In [ ]:
max_match_ts = dbutils.jobs.taskValues.get(taskKey="Extract_Max_ts", key="max_match_ts", debugValue="2026-01-01 00:00:00")
max_delivery_ts = dbutils.jobs.taskValues.get(taskKey="Extract_Max_ts", key="max_delivery_ts", debugValue="2026-01-01 00:00:00")

print(max_match_ts, max_delivery_ts)

In [ ]:
# Read only records with created_at <= max_created_at
df = spark.sql(f"SELECT * FROM bronze.match_loading WHERE ingestion_ts <= '{max_match_ts}'")

In [ ]:
# Select 'city' and 'venue' columns
city_venue_df = df.select("city", "venue")

In [ ]:
from pyspark.sql.functions import col, lit, current_timestamp
# 1. Prepare the DataFrame with ONLY the 7 columns you provide (Remove venue_id)
final_df = city_venue_df.select(
    col("venue").alias("official_name"),
    col("city"),
    lit(None).cast("string").alias("country"),
    lit(None).cast("long").alias("capacity"),
    lit(None).cast("double").alias("latitude"),
    lit(None).cast("double").alias("longitude"),
    current_timestamp().alias("created_at")
)


In [ ]:
final_df = final_df.dropDuplicates(["official_name"])

In [ ]:
existing_official_names = spark.table("silver.dim_venue").select("official_name").distinct()

In [ ]:
final_df = final_df.join(existing_official_names, on="official_name", how="left_anti")

In [ ]:
# 2. Create a temporary view so we can query this DataFrame with SQL
final_df.createOrReplaceTempView("source_new_venues")

In [ ]:
# 3. Use SQL to insert specific columns (skipping venue_id automatically triggers auto-increment)
spark.sql("""
    INSERT INTO silver.dim_venue (
        official_name, 
        city, 
        country, 
        capacity, 
        latitude, 
        longitude, 
        created_at
    )
    SELECT 
        official_name, 
        city, 
        country, 
        capacity, 
        latitude, 
        longitude, 
        created_at 
    FROM source_new_venues
""")

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [ ]:
%sql
SELECT count(DISTINCT official_name) from silver.dim_venue
UNION ALL
SELECT COUNT(*) FROM silver.dim_venue